# Solving the diffusion approximation to the birth-immigration-death process

<!-- <center><font size="+4">Solving the diffusion approximation to the birth-immigration-death process</font></center> -->

## Forward Laplace transform

In [1]:
diffusioneqn (rho, D, s, M) := 
    diff (rho, tau) -
    D * diff (x * rho, x, 2) +
    s * diff (x * rho, x, 1) +
    M * diff (rho, x);

(%o1) diffusioneqn(rho, D, s, M) := diff(rho, tau) - D diff(x rho, x, 2)
                                         + s diff(x rho, x, 1) + M diff(rho, x)

In [2]:
cgfeqn (gam, D, s, M) := 
    diff (gam, tau) +
    (D * theta^2 - s* theta) * diff (gam, theta) -
    M * theta;

(%o2) cgfeqn(gam, D, s, M) := diff(gam, tau)
                                    2
                          + (D theta  - s theta) diff(gam, theta) + (- M) theta

In [3]:
factor(diffusioneqn (rho0(x,tau), D, s, M));

               2
              d                          d
(%o3) - (D x (--- (rho0(x, tau))) - s x (-- (rho0(x, tau)))
                2                        dx
              dx
      d                         d                     d
 - M (-- (rho0(x, tau))) + 2 D (-- (rho0(x, tau))) - ---- (rho0(x, tau))
      dx                        dx                   dtau
 - s rho0(x, tau))

## Characteristic equation

Compute the integral for the implicit solution.

In [4]:
logcontract(integrate(1/(D * thp^2 - s * thp), thp));

                                    D thp - s
                                log(---------)
                                       thp
(%o4)                           --------------
                                      s

Solve the implicit equation.

In [5]:
thetasol : solve ([((D* theta - s) * theta0) / ((D* theta0 - s) * theta) =
    %e^(s*u)], [theta])[1];
theta0sol : solve ([((D* theta - s) * theta0) / ((D* theta0 - s) * theta) =
    %e^(s*u)], [theta0])[1];

                                        s theta0
(%o5)              theta = - -------------------------------
                                              s u
                             (D theta0 - s) %e    - D theta0

                                               s u
                                     s theta %e
(%o6)                theta0 = ---------------------------
                                        s u
                              D theta %e    - D theta + s

Check the solution.

In [6]:
subst(0, u,  thetasol);

(%o7)                           theta = theta0

In [7]:
chareqn(theta) := diff(theta, u) - D * theta^2 + s * theta;

                                                       2
(%o8)        chareqn(theta) := diff(theta, u) - D theta  + s theta

In [8]:
factor(chareqn(part(thetasol, 2)));

(%o9)                                  0

## Homogeneous solution

Re-expressing $u$ in terms of $\tau$ and including the initial condition $P(x,0) = \delta(x-x_0) \Rightarrow \Gamma_{\mathrm{hom}}(\theta, 0)=x_0 \, \theta$, where $\Rightarrow$ indicates taking the logarithm of the Laplace transform, yields

In [9]:
gamhom : x0 * s * theta * %e^(s*tau) / 
    (D * (%e^(s*tau) - 1) * theta + s);

                                  s tau
                              s %e      theta x0
(%o10)                     -------------------------
                                s tau
                           D (%e      - 1) theta + s

In [10]:
factor(gamhom - x0 * subst(tau, u, part(theta0sol, 2)));

(%o11)                                 0

Noting $M=0$ in the homogeneous case

In [11]:
factor(cgfeqn (gamhom, D, s, 0));

(%o12)                                 0

## Inhomogeneous solutions

Find the characteristic which passes through a given point (theta_f, tau_f).

In [12]:
th0sol : solve([thetaf = 
        subst(uf, u, part(thetasol, 2))], [theta0])[1];

                                               s uf
                                    s thetaf %e
(%o13)              theta0 = ------------------------------
                                        s uf
                             D thetaf %e     - D thetaf + s

In [13]:
thfsol : factor(psubst(th0sol, thetasol));

                                              s uf
                                   s thetaf %e
(%o14)        theta = ------------------------------------------
                                 s uf              s u       s u
                      D thetaf %e     - D thetaf %e    + s %e

In [14]:
subst(uf, u, thfsol);

(%o15)                          theta = thetaf

Integrate along that characteristic.

In [15]:
ginhint : integrate((thetaf*s*%e^(uf*s))/
    (s*%e^(s*u)-D*thetaf*%e^(s*u)+D*thetaf*%e^(uf*s)), u);

                            - s uf
                  s uf  u %e
(%o16) s thetaf %e     (----------
                         D thetaf
                            - s uf                      s u              s uf
                          %e       log((D thetaf - s) %e    - D thetaf %e    )
                        - ----------------------------------------------------)
                                               D s thetaf

In [16]:
logcontract(ev(factor(subst(uf, u, ginhint) - subst(0, u, ginhint)), 
        logexpand=super));

                                     s uf
                          D thetaf %e     - D thetaf + s
                      log(------------------------------)
                                        s
(%o17)                -----------------------------------
                                       D

In [17]:
logcontract(ev(factor(subst(uf, u, ginhint) - subst(0, u, ginhint)), 
        logexpand=super));

                                     s uf
                          D thetaf %e     - D thetaf + s
                      log(------------------------------)
                                        s
(%o18)                -----------------------------------
                                       D

Adding back the factor of $M$ provides the solution to the inhomogeneous CGF.

In [18]:
gaminh : (M/D) * log( (D/s) * (%e^(-s * tau) - 1) * theta + 1);

                                 - s tau
                            D (%e        - 1) theta
                      M log(----------------------- + 1)
                                       s
(%o19)                ----------------------------------
                                      D

## Inverse Laplace transform

### Homogeneous solution

We proceed by constructing a change of variable to place the inverse Laplace transform integral into a form enabling the identification of a Bessel function solution. Combining the form of the Laplace transform with the form of `gamhom` we have

In [19]:
schemintgr : x * theta  - a * theta / (b * theta + 1);

                                         a theta
(%o20)                       theta x - -----------
                                       b theta + 1

The following change of variable $\theta \rightarrow \theta'$ will render `shemintgr` a function of $\theta' + \frac{1}{\theta'}$ as expected for a Bessel function

In [20]:
thetaChV: c * thp - 1/b;

                                           1
(%o21)                             c thp - -
                                           b

In [21]:
schemintgrthp: expand(subst(thetaChV, theta, schemintgr));

                                    x      a       a
(%o22)                    c thp x - - + -------- - -
                                    b    2         b
                                        b  c thp

Since we are looking for a form that includes a single factor of $\theta' + \frac{1}{\theta'}$ we look for $c x = \frac{a}{b^2 c}$

In [22]:
cbessel: a^(1/2)*x^(-1/2)/b$
thetaChVab: subst(cbessel, c, thetaChV);
schemintgrelimc: expand(subst(thetaChVab, theta, schemintgr));

                                sqrt(a) thp   1
(%o24)                          ----------- - -
                                 b sqrt(x)    b

                  x    sqrt(a) thp sqrt(x)   sqrt(a) sqrt(x)   a
(%o25)         (- -) + ------------------- + --------------- - -
                  b             b                 b thp        b

To provide $\theta$ in terms of $\theta'$ we substitute the values of $a$ and $b$ given by the homogeneous solution `gamhom`. First we note that if we define $a = x_0  e^{s \tau}$ and $b = \frac{D}{s} (e^{s \tau} -1)$

In [23]:
avalue: x0 * %e^(s*tau);
bvalue: (D/s) * (%e^(s*tau)- 1);

                                    s tau
(%o26)                            %e      x0

                                     s tau
                                D (%e      - 1)
(%o27)                          ---------------
                                       s

that $\mathrm{gamhom} = \frac {a \theta}{b \theta +1}$

In [24]:
factor(gamhom - avalue*theta/(bvalue*theta +1));

(%o28)                                 0

Now we substitute values of $a$ and $b$ into the terms of `schemintgrelimc`


In [25]:
factor(psubst([a = avalue, 
               b = bvalue],
               thetaChVab));

                           s tau
                           -----
                             2
                      s (%e      thp sqrt(x0) - sqrt(x))
(%o29)                ----------------------------------
                                s tau
                           D (%e      - 1) sqrt(x)

In [26]:
factor(psubst([a = avalue, 
               b = bvalue],
               -a/b - x/b));

                                    s tau
                               s (%e      x0 + x)
(%o30)                       - ------------------
                                     s tau
                                D (%e      - 1)

In [27]:
factor(psubst([a = avalue, 
               b = bvalue],
               sqrt(a*x)/b));

                                 s tau
                                 -----
                                   2
                             s %e      sqrt(x x0)
(%o31)                       --------------------
                                    s tau
                               D (%e      - 1)

### Inhomogeneous solution

In [28]:
invinhvarchange : theta = 
    thetap/x - s / (D*(%e^(s * tau) - 1));

                               thetap          s
(%o32)                 theta = ------ - ---------------
                                 x           s tau
                                        D (%e      - 1)

In [29]:
diff(part(invinhvarchange,2), thetap, 1);

                                       1
(%o33)                                 -
                                       x

In [30]:
factor(ratsimp(
  psubst(invinhvarchange, 
         diff(part(invinhvarchange,2), thetap, 1)
         *e^(x*theta)*((D/s) * (%e^(s * tau) - 1) * theta + 1)^(-M/D))
         ));

                                      s tau
                        s x         %e      thetap     thetap
                (- -------------) + -------------- - -----------
                       s tau           s tau           s tau
                   D %e      - D     %e      - 1     %e      - 1
               e
(%o34)         -------------------------------------------------
                               s tau
                          D (%e      - 1) thetap M/D
                         (----------------------)    x
                                   s x

In [31]:
factor(part(%o116,1,2,2)+part(%o116,1,2,3));

part: argument must be a non-atomic expression; found %o116
 -- an error. To debug this try: debugmode(true);


In [32]:
inhdifsol : x^(M/D - 1) * 
    %e^(-s*x/(D * (%e^(s * tau) - 1))) *
    ((D*(%e^(s * tau) - 1)/s)^(-M/D));

                                            s x
                                    - ---------------
                                           s tau
                          M/D - 1     D (%e      - 1)
                         x        %e
(%o36)                   ----------------------------
                                   s tau
                              D (%e      - 1) M/D
                             (---------------)
                                     s

In [33]:
inhdifsol : x^(M/D - 1) * 
    %e^(-s*x/(D * (%e^(s * tau) - 1))) *
    (((%e^(s * tau) - 1))^(-M/D));

                                            s x
                                    - ---------------
                                           s tau
                          M/D - 1     D (%e      - 1)
                         x        %e
(%o37)                   ----------------------------
                                  s tau     M/D
                               (%e      - 1)

In [34]:
factor(expand(diffusioneqn (inhdifsol, D, s, M)));

(%o38)                                 0